In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [8]:
def filtrar_clutch(df):
    """
    Filtra jugadas clutch:
    - periodo >= 4
    - últimos 5 minutos (<= 300s)
    - scoremargin entre -5 y +5
    """
    df = df.copy()
    df["scoremargin"] = pd.to_numeric(df["scoremargin"], errors="coerce")
    df["seconds_remaining"] = np.where(
        df["pctimestring"].str.match(r"^\d{1,2}:\d{2}$", na=False),
        df["pctimestring"].str.split(":").str[0].astype(int) * 60 +
        df["pctimestring"].str.split(":").str[1].astype(int),
        np.nan
    )

    clutch = df[
        (df["period"] >= 4) &
        (df["seconds_remaining"] <= 300) &
        (df["scoremargin"].abs() <= 5)
    ].copy()

    return clutch


In [11]:
cols = [
    "game_id", "eventmsgtype", "period", "pctimestring",
    "scoremargin", "player1_id", "player1_name",
    "homedescription", "visitordescription"
]

dtypes = {
    "game_id": "Int32",
    "eventmsgtype": "UInt8",
    "period": "UInt8",
    "player1_id": "Int32",
    "player1_name": "object",
    "homedescription": "object",
    "visitordescription": "object"
}


In [ ]:
df_1516 = pd.read_csv("NBA_PBP_2015–16.csv", usecols=cols, dtype=dtypes)
df_1516_clutch = filtrar_clutch(df_1516)
print(f"2015–16 → {len(df_1516_clutch)} jugadas clutch")


In [5]:
def filtrar_jugadas_clutch(df):
    df = df.copy()
    df["scoremargin"] = pd.to_numeric(df["scoremargin"], errors="coerce").astype("Int16")

    # Convertir a segundos
    mask_time = df["pctimestring"].str.match(r"^\d{1,2}:\d{2}$", na=False)
    mins = pd.to_numeric(df.loc[mask_time, "pctimestring"].str.split(":").str[0], errors="coerce")
    secs = pd.to_numeric(df.loc[mask_time, "pctimestring"].str.split(":").str[1], errors="coerce")
    df.loc[mask_time, "seconds_remaining"] = mins * 60 + secs

    clutch = df[
        (df["period"] >= 4) &
        (df["seconds_remaining"] <= 300) &
        (df["scoremargin"].abs() <= 5)
    ].copy()

    return clutch


In [6]:
df_clutch = filtrar_jugadas_clutch(df)
